# 3 Data Driven Decision Making with advanced SQL queries

The concept of nested queries and correlated nested queries is introduced and the functions EXISTS and UNION are used to categorize customers, movies, actors, and more.